In [8]:
!pip install Pillow
!pip install "transformers[torch]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

In [10]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText
model_path = "nanonets/Nanonets-OCR-s"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)


def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""
    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

image_path = "/kaggle/input/tableexcel/Excel-table-inserted-in-Worksheet.png"
result = ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=15000)
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<table>
  <tr>
    <td>A</td>
    <td>B</td>
    <td>C</td>
    <td>D</td>
    <td>E</td>
    <td>F</td>
  </tr>
  <tr>
    <td><strong>Store</strong></td>
    <td><strong>Region</strong></td>
    <td><strong>Q1</strong></td>
    <td><strong>Q2</strong></td>
    <td><strong>Q3</strong></td>
    <td><strong>Q4</strong></td>
  </tr>
  <tr>
    <td>Store 1</td>
    <td>East</td>
    <td>8071</td>
    <td>6759</td>
    <td>6105</td>
    <td>1001</td>
  </tr>
  <tr>
    <td>Store 2</td>
    <td>East</td>
    <td>9002</td>
    <td>5898</td>
    <td>2622</td>
    <td>2231</td>
  </tr>
  <tr>
    <td>Store 3</td>
    <td>East</td>
    <td>4439</td>
    <td>9589</td>
    <td>5156</td>
    <td>8679</td>
  </tr>
  <tr>
    <td>Store 4</td>
    <td>West</td>
    <td>9364</td>
    <td>5762</td>
    <td>2700</td>
    <td>1864</td>
  </tr>
  <tr>
    <td>Store 5</td>
    <td>West</td>
    <td>9842</td>
    <td>5091</td>
    <td>4144</td>
    <td>1582</td>
  </tr>
  <tr>
    <td>Store 6</td>
    <td>E

Add path to your image above in image_path

In [11]:
def save_ocr_result_to_file(result_string: str, filename: str, file_format: str = "txt"):
    """Saves OCR result string to a file (txt or md)."""
    full_filename = f"{filename}.{file_format.lower()}"
    try:
        with open(full_filename, "w", encoding="utf-8") as f:
            f.write(result_string)
        print(f"Result saved to '{full_filename}'")
    except Exception as e:
        print(f"Error saving file: {e}")

save_ocr_result_to_file(result, "my_document_output", "txt")

Result saved to 'my_document_output.txt'
